In [54]:
####seleniumを用いたEDINET操作####
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoSuchElementException
import requests
from urllib.request import urlopen
import pandas as pd
import numpy as np
import time
import itertools

In [41]:
##企業データの読み込みとクレンジング
#データの読み込み
company_info1 = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")
index = np.where(np.in1d(company_info1["業種"], np.array(["ETF", "ETN", "REIT", "カントリーF", "インフラF"]))==False)[0]
company_info1 = company_info1.iloc[index]
company_info1.index = np.arange(company_info1.shape[0])
company_info2 = pd.read_csv("D:/Statistics/data/finance_data/stocklist.csv")
company_info2 = company_info2.rename(columns={"銘柄コード": "コード", "市場名": "取引市場", "業種分類": "業種"})
index = np.where(np.in1d(company_info2["業種"], np.array(["REIT銘柄一覧"]))==False)[0]
company_info2 = company_info2.iloc[index]
company_info2.index = np.arange(company_info2.shape[0])

#データの結合
company_info1 = company_info1[["コード", "銘柄名", "取引市場", "業種"]]
company_info2 = company_info2[["コード", "銘柄名", "取引市場", "業種"]]
company_info = pd.concat((company_info1, company_info2), axis=0)
company_info.index = np.arange(company_info.shape[0])

#変数を共通化
company = company_info[["コード", "銘柄名"]].iloc[np.where(company_info[["コード"]].duplicated()==False)[0]]
company_info["銘柄名"] = pd.merge(company_info[["コード"]], company, on="コード", how="left")["銘柄名"]
industry = np.array([['水産・農林業', '卸売業', '建設業', '非鉄金属', '鉱業', '機械', 'サービス業', '金属製品',
                      '情報・通信', '食料品', '医薬品', '不動産業', '陸運業', 'その他金融業', '小売業', 'その他製品',
                      '繊維製品', '電気機器', 'ガラス・土石製品', '証券業', '輸送用機器', '石油・石炭製品', '化学',
                      'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行業', '保険業', '倉庫・運輸関連業', '海運業',
                      '空運業', '電気・ガス業'],
                   ['水産・農林', '卸売', '建設業', '非鉄金属', '鉱業', '機械', 'サービス', '金属製品',
                    '情報通信', '食料品', '医薬品', '不動産', '陸運', 'その他金融', '小売', 'その他製品', 
                    '繊維製品', '電気機器', 'ガラス土石', '証券・先物', '輸送用機器', '石油・石炭', '化学',
                    'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行', '保険', '倉庫・運輸', '海運', 
                    '空運', '電気・ガス']])
industry_vec = np.array(company_info["業種"])
for j in range(industry.shape[1]):
    index = np.where(industry_vec==industry[0, j])[0]
    industry_vec[index] = np.array(industry[1, j], dtype="object")
company_info["業種"] = industry_vec
market = np.array([['東証1部', 'JQS', 'JQG', 'マザーズ', '東証2部', '福証Q', '名証2部', '札証', '福証',
                    '名証1部', '札証A', '名証C', '東証'],
                   ['東証1部', '東証JQS', '東証JQG', 'マザーズ', '東証2部', '福岡Q', '名証2部', '札証', "福証",
                    '名証1部',  '札幌ア', '名古屋セ', '東証']])
market_vec = np.array(company_info["取引市場"])
for j in range(market.shape[1]):
    index = np.where(market_vec==market[0, j])[0]
    market_vec[index] = np.array(market[1, j], dtype="object")
company_info["取引市場"] = market_vec

#重複を削除
company_info = company_info.iloc[np.where(company_info["コード"].duplicated()==False)[0]]
company_info.index = np.arange(company_info.shape[0])
n = company_info.shape[0]

In [51]:
#スクレイピング対象urlを作成
url_sample = "https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?uji.verb=W1E63020CXW1E6A020DSPSch&uji.\
bean=ee.bean.parent.EECommonSearchBean&PID=W1E63020&TID=W1E63021&SESSIONKEY=1562924847587&lgKbn=2&pkbn=0&skbn=0\
&dskb=&dflg=0&iflg=0&preId=1&row=100&idx=0&syoruiKanriNo=&sec=&scc=1301&shb=&snm=&spf1=1&spf2=1&iec=&icc=&inm=&\
spf3=1&fdc=&fnm=&spf4=1&spf5=2&otd=120&otd=140&otd=160&cal=1&era=R&yer=&mon=&psr=1&pfs=5"

url = np.repeat(url_sample, n)
code = "scc={code}"
for i in range(n):
    url[i] = url_sample.replace("scc=1301", code.format(code=company_info["コード"].iloc[i]))
company_info["url"] = url

In [55]:
#seleniumでブラウザを読み込み
browser = webdriver.Chrome()

#XBRLをダウンロード
for i in range(n):    
    print([i, company_info["銘柄名"].iloc[i]])
    browser.get(url[i])   #URLを検索
    browser.execute_script("javascript:downloadConfirmFile('W1E63021CXP002002DSPXbrl', \
    'ee.bean.parent.EECommonSearchBean', 'W1E63021', 'W1E63021', 'lgKbn=2', '/E01EW/download?1562924986438');")
    Alert(browser).accept()
    
    #待ち時間
    time.sleep(5)

[0, '極洋']
[1, '日本水産']
[2, 'マルハニチロ']
[3, 'ホウスイ']
[4, 'カネコ種苗']
[5, 'サカタのタネ']
[6, 'ホクト']
[7, '秋川牧園']
[8, 'アクシーズ']
[9, 'ホーブ']
[10, 'ベルグアース']
[11, 'ホクリヨウ']
[12, 'ルーデン・HD']
[13, 'エムビーエス']
[14, 'ウエストHD']
[15, 'サムシングHD']
[16, '桧家HD']
[17, 'ショーボンドHD']
[18, 'ミライト・HD']
[19, 'インターライフHD']
[20, 'タマホーム']
[21, 'サンヨーホームズ']
[22, '日本アクア']
[23, 'ファーストCorp.']
[24, 'エスケーホーム']
[25, 'ベステラ']
[26, 'ＪＥＳＣＯHD']
[27, 'インベスターズクラウド']
[28, 'フィット']
[29, '岐阜造園']
[30, '安江工務店']
[31, '中外鉱業']
[32, '住石HD']
[33, '日鉄鉱業']
[34, '三井松島産業']
[35, '国際石油開発帝石']
[36, '日本海洋掘削']
[37, '石油資源開発']
[38, 'Ｋ＆Ｏエナジーグループ']
[39, '省電舎HD']
[40, 'ダイセキ環境ソリューション']
[41, '第一カッター興業']
[42, '明豊ファシリティワークス']
[43, '美樹工業']
[44, '安藤ハザマ']
[45, '東急建設']
[46, 'コムシスHD']
[47, 'ミサワホーム']
[48, '日本電技']
[49, 'シンクレイヤ']
[50, 'ビーアールHD']
[51, 'ミサワホーム中国']
[52, '麻生フオームクリート']
[53, '北弘電社']
[54, 'オーテック']
[55, '三井金属エンジニアリング']
[56, 'ＮＩＴＴＯＨ']
[57, 'シード平和']
[58, 'コーアツ工業']
[59, 'クレアHD']
[60, '太洋基礎工業']
[61, '高松コンストラクショングループ']
[62, '工藤建設']
[63, '東建Corp.']
[64, 'ソネック']
[65, '藤田エンジニアリング']
[66

[523, 'セイヒョー']
[524, 'ヨコレイ']
[525, '東洋水産']
[526, 'ジェーシー・コムサ']
[527, '日東ベスト']
[528, 'イートアンド']
[529, '大冷']
[530, 'ヨシムラ・フード・HD']
[531, '日本食品化工']
[532, '石井食品']
[533, '日清食品HD']
[534, '永谷園HD']
[535, '石垣食品']
[536, '太陽化学']
[537, 'シノブフーズ']
[538, '一正蒲鉾']
[539, 'オーケー食品工業']
[540, 'あじかん']
[541, 'フジッコ']
[542, 'ロック・フィールド']
[543, '旭松食品']
[544, 'ＪＴ']
[545, 'ケンコーマヨネーズ']
[546, '仙波糖化工業']
[547, '大森屋']
[548, 'わらべや日洋HD']
[549, 'マルタイ']
[550, 'なとり']
[551, 'サトウ食品工業']
[552, 'イフジ産業']
[553, 'ピックルスCorp.']
[554, '篠崎屋']
[555, 'ＡＦＣ-ＨＤアムスライフサイエンス']
[556, 'ＲＩＺＡＰグループ']
[557, 'ファーマフーズ']
[558, '北の達人Corp.']
[559, 'ユーグレナ']
[560, '片倉工業']
[561, 'グンゼ']
[562, 'ヒューリック']
[563, '神栄']
[564, '価値開発']
[565, 'バナーズ']
[566, 'アプライド']
[567, 'パシフィックネット']
[568, 'ラサ商事']
[569, 'クリエイト']
[570, 'アルペン']
[571, 'ハブ']
[572, 'ラクーン']
[573, 'ゴルフ・ドゥ']
[574, 'クオール']
[575, 'ケイティケイ']
[576, 'アルコニックス']
[577, '神戸物産']
[578, 'ソリトンシステムズ']
[579, 'ビューティ花壇']
[580, 'セキュアヴェイル']
[581, 'カワサキ']
[582, 'ジンズ']
[583, 'ＴＲＵＣＫ-ＯＮＥ']
[584, 'ビックカメラ']
[585, 'ＤＣＭHD']
[586, 'ペッパーフードサ

[1013, '大和コンピューター']
[1014, 'ＳＲＡHD']
[1015, 'Ｍｉｎｏｒｉソリューションズ']
[1016, 'アクロディア']
[1017, 'メディアファイブ']
[1018, 'リミックスポイント']
[1019, 'システムインテグレータ']
[1020, 'ギガプライズ']
[1021, '朝日ネット']
[1022, 'ｅＢＡＳＥ']
[1023, 'アバント']
[1024, 'アドソル日進']
[1025, 'ＯＤＫソリューションズ']
[1026, 'パス']
[1027, 'ジーダット']
[1028, 'ネクストジェン']
[1029, 'フリービット']
[1030, 'コムチュア']
[1031, 'アイフリークモバイル']
[1032, 'パシフィックシステム']
[1033, 'データ・アプリケーション']
[1034, '日本テクノ・ラボ']
[1035, 'ＮＴＴデータイントラマート']
[1036, '日本一ソフトウェア']
[1037, 'サイバーコム']
[1038, 'インフォテリア']
[1039, 'アイル']
[1040, 'Ａｂａｌａｎｃｅ']
[1041, 'ラック']
[1042, 'ユビキタス']
[1043, '王子HD']
[1044, '日本製紙']
[1045, '三菱製紙']
[1046, '北越紀州製紙']
[1047, '中越パルプ工業']
[1048, '巴川製紙所']
[1049, '大王製紙']
[1050, 'ニッポン高度紙工業']
[1051, '岡山製紙']
[1052, 'ハビックス']
[1053, '阿波製紙']
[1054, 'クラウドワークス']
[1055, 'マークラインズ']
[1056, 'メディカル・データ・ビジョン']
[1057, 'ｇｕｍｉ']
[1058, 'カヤック']
[1059, 'データセクション']
[1060, 'ＡＬＢＥＲＴ']
[1061, 'シリコンスタジオ']
[1062, 'コラボス']
[1063, 'ショーケース・ティービー']
[1064, 'エムケイシステム']
[1065, 'Ａｉｍｉｎｇ']
[1066, 'モバイルファクトリー']
[1067, 'ｓＭｅｄｉｏ']
[1068, 'ＪＩＧ-ＳＡＷ'

[1481, '山田コンサルティンググループ']
[1482, '富士通ビー・エス・シー']
[1483, 'オリコン']
[1484, 'セントラルスポーツ']
[1485, 'パラカ']
[1486, 'ＩＳＩＤ']
[1487, 'ＡＣＣＥＳＳ']
[1488, 'ネクストウェア']
[1489, '東映アニメーション']
[1490, 'デジタルガレージ']
[1491, 'ＥＭシステムズ']
[1492, 'メディアシーク']
[1493, 'ウェザーニューズ']
[1494, 'ＣＩＪ']
[1495, 'ビジネス・ワンHD']
[1496, '東洋ビジネスエンジニアリング']
[1497, '日本エンタープライズ']
[1498, 'ＪＦＥシステムズ']
[1499, 'ぱど']
[1500, 'キャリアバンク']
[1501, 'シダックス']
[1502, 'スペースシャワーネットワーク']
[1503, 'ＷＯＷＯＷ']
[1504, 'トライアイズ']
[1505, 'スカラ']
[1506, 'インテリジェント_ウェイブ']
[1507, 'フルキャストHD']
[1508, 'エン・ジャパン']
[1509, 'メディシノバ']
[1510, '富士フイルムHD']
[1511, 'コニカミノルタ']
[1512, '資生堂']
[1513, 'ライオン']
[1514, '高砂香料工業']
[1515, 'マンダム']
[1516, 'アイビー化粧品']
[1517, 'ミルボン']
[1518, '日本色材工業研究所']
[1519, 'ファンケル']
[1520, 'コーセー']
[1521, 'コタ']
[1522, 'シーズ・HD']
[1523, 'ハーバー研究所']
[1524, 'シーボン']
[1525, 'ポーラ・オルビスHD']
[1526, 'ノエビアHD']
[1527, 'アジュバンコスメジャパン']
[1528, 'エステー']
[1529, 'エス・ディー・エス_バイオテック']
[1530, 'アグロ_カネショウ']
[1531, 'コニシ']
[1532, 'ヤスハラケミカル']
[1533, '長谷川香料']
[1534, 'ケミプロ化成']
[1535, '互応化学工業']
[1536, '星光ＰＭＣ

[1962, 'エヌ・ピー・シー']
[1963, 'ニューフレアテクノロジー']
[1964, '藤商事']
[1965, '平田機工']
[1966, 'ペガサスミシン製造']
[1967, 'マルマエ']
[1968, '妙徳']
[1969, 'タツモ']
[1970, 'ゼネラルパッカー']
[1971, 'ナブテスコ']
[1972, '三井海洋開発']
[1973, 'ニッセイ']
[1974, 'レオン自動機']
[1975, 'ＳＭＣ']
[1976, '新川']
[1977, 'ナビタス']
[1978, 'ホソカワミクロン']
[1979, 'ユニオンツール']
[1980, '瑞光']
[1981, '前田製作所']
[1982, 'オイレス工業']
[1983, '日精エー・エス・ビー機械']
[1984, '靜甲']
[1985, 'サトーHD']
[1986, '技研製作所']
[1987, '日本エアーテック']
[1988, 'カワタ']
[1989, '日精樹脂工業']
[1990, 'オカダアイヨン']
[1991, '富士変速機']
[1992, '鉱研工業']
[1993, 'ワイエイシイHD']
[1994, '神鋼環境ソリューション']
[1995, 'アピックヤマダ']
[1996, 'コマツ']
[1997, '住友重機械工業']
[1998, 'ササクラ']
[1999, '日立建機']
[2000, '日工']
[2001, 'サンセイ']
[2002, '巴工業']
[2003, '井関農機']
[2004, 'フロイント産業']
[2005, '石井工作研究所']
[2006, 'ＴＯＷＡ']
[2007, '丸山製作所']
[2008, '北川鉄工所']
[2009, 'シンニッタン']
[2010, 'タクミナ']
[2011, 'ローツェ']
[2012, 'ハーモニック・ドライブ・システムズ']
[2013, 'タカキタ']
[2014, 'クボタ']
[2015, '北川精機']
[2016, '荏原実業']
[2017, '東洋エンジニアリング']
[2018, '三菱化工機']
[2019, '月島機械']
[2020, '帝国電機製作所']
[2021, '明治機械']
[2022, '東京機

[2448, '東京ＴＹFG']
[2449, '今村証券']
[2450, 'ＧＭＯフィナンシャルHD']
[2451, '九州FG']
[2452, 'かんぽ生命保険']
[2453, 'ゆうちょ銀行']
[2454, 'あんしん保証']
[2455, '富山第一銀行']
[2456, 'ヒロセ通商']
[2457, 'コンコルディア・FG']
[2458, 'ジェイリース']
[2459, '西日本フィナンシャルHD']
[2460, 'マーキュリアインベストメント']
[2461, 'イントラスト']
[2462, '日本モーゲージサービス']
[2463, 'Ｃａｓａ']
[2464, 'アルヒ']
[2465, 'プレミアグループ']
[2466, '日産自動車']
[2467, 'いすゞ自動車']
[2468, 'トヨタ自動車']
[2469, '日野自動車']
[2470, 'カネミツ']
[2471, '三菱自動車']
[2472, 'エフテック']
[2473, 'レシップHD']
[2474, 'ＧＭＢ']
[2475, 'ファルテック']
[2476, 'テイン']
[2477, '田中精密工業']
[2478, 'エッチ・ケー・エス']
[2479, '武蔵精密工業']
[2480, '日産車体']
[2481, '新明和工業']
[2482, '極東開発工業']
[2483, 'アスカ']
[2484, 'デイトナ']
[2485, 'ユタカ技研']
[2486, '日信工業']
[2487, 'トピー工業']
[2488, '東京ラヂエーター製造']
[2489, 'ティラド']
[2490, '曙ブレーキ工業']
[2491, 'タチエス']
[2492, 'ＮＯＫ']
[2493, 'フタバ産業']
[2494, 'ＫＹＢ']
[2495, '市光工業']
[2496, '大同メタル工業']
[2497, 'プレス工業']
[2498, 'ミクニ']
[2499, '尾張精機']
[2500, '太平洋工業']
[2501, 'ケーヒン']
[2502, 'ユニバンス']
[2503, '桜井製作所']
[2504, '河西工業']
[2505, 'アイシン精機']
[2506, '富士機工']
[2507, 'マツダ']
[250

[2949, 'ニチモウ']
[2950, '極東貿易']
[2951, 'イワキ']
[2952, '兼松エレクトロニクス']
[2953, '三愛石油']
[2954, '稲畑産業']
[2955, 'ＧＳＩクレオス']
[2956, '明和産業']
[2957, 'クワザワ']
[2958, '堀田丸正']
[2959, 'キムラタン']
[2960, 'ヤマエ久野']
[2961, 'ゴールドウイン']
[2962, 'ユニ・チャーム']
[2963, 'デサント']
[2964, 'ムーンバット']
[2965, '中央自動車工業']
[2966, 'キング']
[2967, '三栄Corp.']
[2968, '川辺']
[2969, 'ワキタ']
[2970, 'ヤマト_インターナショナル']
[2971, '東邦HD']
[2972, 'サンゲツ']
[2973, 'ミツウロコグループHD']
[2974, 'シナネンHD']
[2975, '伊藤忠エネクス']
[2976, 'ゼット']
[2977, 'サンリオ']
[2978, 'サンワテクノス']
[2979, '三京化成']
[2980, 'ナガホリ']
[2981, 'リョーサン']
[2982, '新光商事']
[2983, 'トーホー']
[2984, 'ラピーヌ']
[2985, '電響社']
[2986, '中部水産']
[2987, 'トミタ']
[2988, '上原成商事']
[2989, '三信電気']
[2990, '東陽テクニカ']
[2991, 'ソマール']
[2992, 'モスフードサービス']
[2993, '加賀電子']
[2994, '三益半導体工業']
[2995, '都築電気']
[2996, 'ソーダニッカ']
[2997, '立花エレテック']
[2998, '木曽路']
[2999, 'ＳＲＳHD']
[3000, '千趣会']
[3001, 'タカキュー']
[3002, 'リテールパートナーズ']
[3003, 'ケーヨー']
[3004, 'マックスバリュ中部']
[3005, '上新電機']
[3006, '日本瓦斯']
[3007, 'ロイヤルHD']
[3008, '東天紅']
[3009, 'いなげや']
[3010, '島忠']
[3

[3439, '日本電信電話']
[3440, 'ＫＤＤＩ']
[3441, '光通信']
[3442, '沖縄セルラー電話']
[3443, 'ＮＴＴドコモ']
[3444, 'エムティーアイ']
[3445, 'エム・エイチ・グループ']
[3446, 'ベルパーク']
[3447, 'トーシン']
[3448, 'フォーバルテレコム']
[3449, 'サカイHD']
[3450, 'ＧＭＯインターネット']
[3451, 'アイドママーケティングコミュニケーション']
[3452, 'アルファポリス']
[3453, 'カドカワ']
[3454, '学研HD']
[3455, '文溪堂']
[3456, 'ゼンリン']
[3457, '昭文社']
[3458, '中央経済社HD']
[3459, 'ＳＥHD・アンド・インキュベーションズ']
[3460, 'インプレスHD']
[3461, '東京電力HD']
[3462, '中部電力']
[3463, '関西電力']
[3464, '中国電力']
[3465, '北陸電力']
[3466, '東北電力']
[3467, '四国電力']
[3468, '九州電力']
[3469, '北海道電力']
[3470, '沖縄電力']
[3471, 'Ｊ-ＰＯＷＥＲ']
[3472, 'エフオン']
[3473, 'イーレックス']
[3474, 'レノバ']
[3475, '東京ガス']
[3476, '大阪ガス']
[3477, '東邦ガス']
[3478, '北海道ガス']
[3479, '広島ガス']
[3480, '西部ガス']
[3481, '北陸ガス']
[3482, '京葉瓦斯']
[3483, '静岡ガス']
[3484, 'メタウォーター']
[3485, 'アイネット']
[3486, '松竹']
[3487, '東宝']
[3488, 'エイチ・アイ・エス']
[3489, '東映']
[3490, 'ウィルソン・ラーニング_ワールドワイド']
[3491, 'ラックランド']
[3492, 'ＮＴＴデータ']
[3493, '共立メンテナンス']
[3494, 'イチネンHD']
[3495, '建設技術研究所']
[3496, 'スペース']
[3497, '長大']
[3498, 'セレ

In [50]:
i = 0


[0, '極洋']
